# Generate BitMasks

The purpose of this notebook is to generate bitmasks to be used as filters, using the BAMBOO algorithm.

In [1]:
def generate_bitmaps(
    total_length: str, mask_length: str, mask_characters_list: list
) -> list:
    if total_length % mask_length != 0:
        raise ValueError(f"Total length must be a multiple of {mask_length}.")

    num_bitmaps = total_length // mask_length

    bitmaps = [
        "0" * (i * mask_length)
        + mask_characters_list[0] * (mask_length // 2)
        + mask_characters_list[1] * (mask_length // 2)
        + "0" * ((num_bitmaps - i - 1) * mask_length)
        for i in range(num_bitmaps)
    ]
    return bitmaps

In [2]:
total_length = 1024
mask_length = 8
mask_characters = ["1", "-1"]
bitmaps = generate_bitmaps(total_length, mask_length, mask_characters)
for bitmap in bitmaps:
    print(bitmap)

1111-1-1-1-10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [3]:
len(bitmaps)

128

In [4]:
def generate_specific_bitmask_patterns(L, lengths=[8]):
    filters = []

    def create_specific_patterns(length):
        """Helper function to create specific patterns"""
        half_len = length // 2

        # Pattern 1: First half -1, last half 1
        pattern1 = ["N"] * half_len + [1] * (length - half_len)

        # Pattern 2: First half 1, last half -1
        pattern2 = [1] * half_len + ["N"] * (length - half_len)

        # Pattern 3: All 1
        pattern3 = [1] * length

        # Pattern 4: All -1
        pattern4 = ["N"] * length

        # Pattern 5: Alternating -1 and 1
        pattern5 = ["N" if i % 2 == 0 else 1 for i in range(length)]

        # Pattern 6: Alternating 1 and -1
        pattern6 = [1 if i % 2 == 0 else "N" for i in range(length)]

        return [pattern1, pattern2, pattern3]

    for length in lengths:
        num_windows = L // length  # Number of full windows we can fit in L

        for i in range(num_windows):
            # Create a base mask of 0s with length `L`
            base_mask = [0] * L

            # Define the position of the window
            start_pos = i * length
            end_pos = start_pos + length

            # Get all specific patterns for the current window length
            patterns = create_specific_patterns(length)

            for pattern in patterns:
                # Create a mask with the current pattern
                mask = base_mask[:]

                for j in range(length):
                    if start_pos + j < L:
                        mask[start_pos + j] = pattern[j]

                # Convert the mask list to a string
                filters.append("".join(map(str, mask)))

    return filters


# Example usage:
L = 2288
bitmask_filters = generate_specific_bitmask_patterns(L)
for f in bitmask_filters:
    print(f)

...


In [5]:
len(bitmask_filters)

858

In [6]:
type(bitmask_filters)

list

In [7]:
import csv

In [8]:
def save_patterns_to_csv(patterns, filename):
    with open(filename, "w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        for pattern in patterns:
            csvwriter.writerow([pattern])

In [9]:
save_patterns_to_csv(
    bitmask_filters, "../../data/filters/bitmask_patterns_sliding_window.csv"
)

## Printing Experiments

In [10]:
import numpy as np

In [11]:
data = np.array(
    [
        ["0", "0", "0", "1", "0", "0", "0"],
        ["0", "0", "0", "1", "1", "1", "0"],
        ["0", "0", "0", "1", "0", "1", "0"],
        ["0", "0", "0", "1", "0", "0", "0"],
        ["0", "0", "0", "0", "1", "0", "1"],
    ]
)

In [12]:
filter = np.array(["-1", "1", "-1", "1", "-1", "-1", "-1"])

In [13]:
len(data)

5

In [14]:
len(filter)

7

In [15]:
filtered = np.multiply(data.astype(int), filter.T.astype(int))

In [16]:
filtered

array([[ 0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  1, -1, -1,  0],
       [ 0,  0,  0,  1,  0, -1,  0],
       [ 0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0, -1]])

In [17]:
# Sum the rows of filtered
row_sums = np.sum(filtered, axis=1)

# Create a column vector with the results
column_vector = np.reshape(row_sums, (-1, 1))

In [18]:
row_sums

array([ 1, -1,  0,  1, -2])

In [19]:
column_vector

array([[ 1],
       [-1],
       [ 0],
       [ 1],
       [-2]])

In [20]:
a = "-1-111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

In [21]:
a = np.array([0, 1, 2, 3, 4, 5, 6])
b = np.array([0, 1, 2, 3, 4, 5, 6])

In [22]:
a * b

array([ 0,  1,  4,  9, 16, 25, 36])

In [23]:
np.multiply(a, b)

array([ 0,  1,  4,  9, 16, 25, 36])

In [24]:
len(
    "11111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"
)

2168

In [25]:
def format_non_zero_part(s):
    # Find the index of the first non-zero character
    first_non_zero = -1
    for i in range(len(s)):
        if s[i] != "0":
            first_non_zero = i
            break

    # Find the index of the last non-zero character
    last_non_zero = -1
    for i in range(len(s) - 1, -1, -1):
        if s[i] != "0":
            last_non_zero = i
            break

    if first_non_zero == -1:
        # If there are no non-zero characters in the string
        return f"0[{len(s)}]"

    # Number of zeros before the first non-zero character
    zeros_before = first_non_zero
    # Number of zeros after the last non-zero character
    zeros_after = len(s) - last_non_zero - 1

    # The non-zero part of the string
    non_zero_part = s[first_non_zero : last_non_zero + 1]

    result = f"0[{zeros_before}] {non_zero_part} 0[{zeros_after}]"
    return result


# Example usage
s = "0000000000333-100000"
formatted_string = format_non_zero_part(s)
print(formatted_string)

0[10] 333-1 0[5]
